# Results of complementation of Büchi automata

In [1]:
from ltlcross_wrapper import ResAnalyzer, gather_cumulative, gather_mins
import pandas as pd

In [2]:
from tools import tools, benchmark_names

### Visualization of the cumulative data over all benchmarks
For each benchmark, we list the cumulative numbers of states for each tool. The best value for each benchmark is highlighted by green background. The `yes` prefix in the tool names means that the Spot simplifications were applied on the results of the tools (were not disabled for seminator). By setting
```python
tools = None
```
you can display results also before the simplifications of Spot were applied, and, moreover, results for `autfilt` configured to produce _deterministic parity automata_ (`autfilt_DPA`) will be also available.

The tools are:
* `autfilt` : Spot's complementation that produces TGBA output
* `buechic` : Buechic from [ROLL](https://iscasmc.ios.ac.cn/roll/doku.php) library, based on automata learning techniques
* `goal#fri` : The Fribourg complementation plugin for GOAL
* `goal#pit` : The complementation of GOAL based on Piterman's determinization (variant of Safra's construction) and conversion to NBA.
* `ncsb` : Complementation in `seminator` that is a transition-based version of the [NCSB algorithm](https://www.fi.muni.cz/~xblahoud/publications/tacas2016preprint.pdf) for complementation of semi-deterministic automata. `#best` indicates, that the better of 2 outputs were chosen (the default behaviour of Seminator). In the extended results (with `tools=None`) we have the variants:
  - `#pldi` the new version of the algorithm implemented in Seminator, it is based on this [PLDI'18 paper](https://dl.acm.org/doi/10.1145/3192366.3192405).
  - `#spot` the algorithm as was already implemented in Spot
  
The benchmarks start with translation of either `random` formulas or formulas from `literature` by `ltl2tgba`. The suffix `_det` indicates that `ltl2tgba` created automata, that are already deterministic, `_sd` stands for semi-deterministic (but not deterministic), and `_nd` represent automata that are not even semi-deterministic.

In [3]:
names = benchmark_names
tools = ["yes.autfilt","yes.ncsb#best","yes.goal#fri","yes.goal#pit","yes.buechenic"]

In [4]:
benchmarks = {}
for name in names:
    b = ResAnalyzer(f"data/{name}.csv", tool_set=tools, cols=["states","time","acc","transitions","edges"])
    b.name = name
    b.orig_count = len(b.values)
    b.clean_count = len(b.values.dropna())
    benchmarks[name] = b

In [5]:
gather_cumulative(benchmarks)

,literature_det,literature_sd,literature_nd,random_det,random_sd,random_nd
tool,,,,,,
yes.autfilt,611,191,188,2439,2892,5492
yes.buechenic,1388,866,309,3638,5766,6330
yes.goal#fri,625,322,189,2490,3367,5305
yes.goal#pit,615,872,243,2451,3389,7628
yes.ncsb#best,622,240,186,2473,2828,5003


## Minimal automata
The follwing table shows for how many formulas each tool produces automaton that has the smallest number of states. The minimum ranges over the considered tools. The number in `min hits` shows how many times the same size as the smallest automaton was achieved. The number in `unique min hits` counts only cases where the given tool is the only tool with such a small automaton.

In [6]:
gather_mins(benchmarks)

### Time in seconds

In [7]:
gather_cumulative(benchmarks, col="time")

,literature_det,literature_sd,literature_nd,random_det,random_sd,random_nd
tool,,,,,,
yes.autfilt,8.57435,2.5377,14.1334,25.9194,28.1235,155.879
yes.buechenic,980.031,239.804,672.446,1136.08,1240.11,3130.46
yes.goal#fri,862.555,420.565,332.866,2347.04,2483.33,3709.36
yes.goal#pit,571.794,392.876,257.134,1610.62,2001.39,2964.27
yes.ncsb#best,10.2817,2.85127,2.68155,29.3337,30.7912,384.036


## Closer look at benchmark by input automaton type

In [8]:
for t in ["nd", "sd", "det"]:
    bench = {n: b for n, b in benchmarks.items() if n[-3:].find(t) >= 0}
    display(t, gather_cumulative(bench, tools))
    for b in bench.values():
        display(b.cross_compare(tools, include_fails=True))
    print("\n\n\n")

'nd'

,literature_nd,random_nd
tool,,
yes.autfilt,188,5492
yes.buechenic,309,6330
yes.goal#fri,189,5305
yes.goal#pit,243,7628
yes.ncsb#best,186,5003


,yes.autfilt,yes.ncsb#best,yes.goal#fri,yes.goal#pit,yes.buechenic,V
yes.autfilt,nan,7,5,11,18,41
yes.ncsb#best,12,nan,6,15,18,51
yes.goal#fri,12,9,nan,15,19,55
yes.goal#pit,6,2,1,nan,16,25
yes.buechenic,2,2,1,4,nan,9


,yes.autfilt,yes.ncsb#best,yes.goal#fri,yes.goal#pit,yes.buechenic,V
yes.autfilt,nan,195,223,350,374,1142
yes.ncsb#best,253,nan,254,369,407,1283
yes.goal#fri,230,213,nan,361,391,1195
yes.goal#pit,105,88,95,nan,286,574
yes.buechenic,125,91,109,213,nan,538


'sd'

...

,yes.autfilt,yes.ncsb#best,yes.goal#fri,yes.goal#pit,yes.buechenic,V
yes.autfilt,nan,108,237,210,483,1038
yes.ncsb#best,142,nan,246,215,486,1089
yes.goal#fri,109,91,nan,140,473,813
yes.goal#pit,72,56,161,nan,451,740
yes.buechenic,13,14,24,48,nan,99


'det'

,literature_det,random_det
tool,,
yes.autfilt,611,2439
yes.buechenic,1388,3638
yes.goal#fri,625,2490
yes.goal#pit,615,2451
yes.ncsb#best,622,2473


,yes.autfilt,yes.ncsb#best,yes.goal#fri,yes.goal#pit,yes.buechenic,V
yes.autfilt,nan,14,13,10,152,189
yes.ncsb#best,14,nan,8,4,148,174
yes.goal#fri,15,13,nan,5,152,185
yes.goal#pit,15,11,6,nan,152,184
yes.buechenic,0,4,0,0,nan,4


,yes.autfilt,yes.ncsb#best,yes.goal#fri,yes.goal#pit,yes.buechenic,V
yes.autfilt,nan,64,40,42,494,640
yes.ncsb#best,9,nan,30,16,489,544
yes.goal#fri,20,57,nan,36,492,605
yes.goal#pit,11,48,21,nan,492,572
yes.buechenic,0,5,2,2,nan,9


In [9]:
b = benchmarks["random_nd"]

In [15]:
b.bokeh_scatter_plot("yes.goal#fri","yes.ncsb#best",alpha=.5, include_equal=True)

Loading BokehJS ...

Figure(id='1398', ...)

In [16]:
b.bokeh_scatter_plot("yes.autfilt","yes.ncsb#best",alpha=.5)

Loading BokehJS ...

Figure(id='1550', ...)

In [17]:
b.bokeh_scatter_plot("yes.goal#pit","yes.ncsb#best",alpha=.5)

Loading BokehJS ...

Figure(id='1712', ...)

## Without simplifications of Spot
GOAL#pit runs removing dead and unreachable states, Buechenic probably does not create such states. The rest of the tools does not remove them.

We can observe that Fribourg generates large ammount of unnecessary states before simplifications.

In [18]:
no_tools = [t.replace("yes","no") for t in tools]

In [20]:
gather_cumulative(benchmarks, tool_set=no_tools)

,literature_det,literature_sd,literature_nd,random_det,random_sd,random_nd
tool,,,,,,
no.autfilt,613,257,205,2442,3578,7604
no.buechenic,1635,915,313,4710,6903,7627
no.goal#fri,1296,2518,2946,5059,17369,56448
no.goal#pit,770,1033,1078,2915,8494,21103
no.ncsb#best,804,439,562,3060,6446,20750


## Minimal automata
The follwing table shows for how many formulas each tool produces automaton that has the smallest number of states. The minimum ranges over the considered tools. The number in `min hits` shows how many times the same size as the smallest automaton was achieved. The number in `unique min hits` counts only cases where the given tool is the only tool with such a small automaton.

In [21]:
gather_mins(benchmarks, tool_set=no_tools)

### Time in seconds

In [23]:
gather_cumulative(benchmarks, col="time", tool_set=no_tools)

,literature_det,literature_sd,literature_nd,random_det,random_sd,random_nd
tool,,,,,,
no.autfilt,7.94785,2.59661,1.14557,25.8379,27.3733,32.8517
no.buechenic,1005.34,234.268,700.125,1134.28,1222.84,3201.37
no.goal#fri,857.512,421.558,315.609,2364.8,2478.42,3531.84
no.goal#pit,585.173,431.692,304.58,1605.61,1990.61,2779.18
no.ncsb#best,1.75239,0.712357,0.857351,5.64361,29.1077,15.3791


In [25]:
for t in ["nd", "sd", "det"]:
    bench = {n: b for n, b in benchmarks.items() if n[-3:].find(t) >= 0}
    display(t, gather_cumulative(bench, no_tools))
    for b in bench.values():
        display(b.cross_compare(no_tools))
    print("\n\n\n")

'nd'

,literature_nd,random_nd
tool,,
yes.autfilt,188,5492
yes.buechenic,309,6330
yes.goal#fri,189,5305
yes.goal#pit,243,7628
yes.ncsb#best,186,5003


,no.autfilt,no.ncsb#best,no.goal#fri,no.goal#pit,no.buechenic,V
no.autfilt,nan,19,20,18,18,75
no.ncsb#best,1,nan,17,8,6,32
no.goal#fri,0,3,nan,0,5,8
no.goal#pit,2,12,19,nan,8,41
no.buechenic,2,14,15,12,nan,43


,no.autfilt,no.ncsb#best,no.goal#fri,no.goal#pit,no.buechenic,V
no.autfilt,nan,443,500,452,364,1759
no.ncsb#best,56,nan,477,330,212,1075
no.goal#fri,0,23,nan,1,26,50
no.goal#pit,42,160,496,nan,163,861
no.buechenic,135,288,474,336,nan,1233


'sd'

...

,no.autfilt,no.ncsb#best,no.goal#fri,no.goal#pit,no.buechenic,V
no.autfilt,nan,372,500,446,471,1789
no.ncsb#best,50,nan,500,294,369,1213
no.goal#fri,0,0,nan,0,108,108
no.goal#pit,27,107,500,nan,315,949
no.buechenic,28,131,390,176,nan,725


'det'

,literature_det,random_det
tool,,
yes.autfilt,611,2439
yes.buechenic,1388,3638
yes.goal#fri,625,2490
yes.goal#pit,615,2451
yes.ncsb#best,622,2473


,no.autfilt,no.ncsb#best,no.goal#fri,no.goal#pit,no.buechenic,V
no.autfilt,nan,141,152,111,152,556
no.ncsb#best,0,nan,150,55,144,349
no.goal#fri,0,0,nan,0,90,90
no.goal#pit,0,77,152,nan,152,381
no.buechenic,0,7,61,0,nan,68


,no.autfilt,no.ncsb#best,no.goal#fri,no.goal#pit,no.buechenic,V
no.autfilt,nan,449,500,432,500,1881
no.ncsb#best,0,nan,490,72,477,1039
no.goal#fri,0,8,nan,6,208,222
no.goal#pit,0,144,492,nan,498,1134
no.buechenic,0,22,287,2,nan,311


## Errors
We can observe several timeouts (120s) and 2 parsing errors on GOAL configurations. These occur on 2 formulas equivalent to false. GOAL does not see the automata as Büchi.

In [26]:
for name, b in benchmarks.items():
    display(name, b.get_error_counts())

'literature_det'

,timeout,parse error,incorrect,crash,no output
no.buechenic,4,0,0,0,0
yes.buechenic,4,0,0,0,0


'literature_sd'

,timeout,parse error,incorrect,crash,no output
no.buechenic,1,0,0,0,0
no.goal#fri,1,0,0,0,0
no.goal#pit,2,0,0,0,0
yes.buechenic,1,0,0,0,0
yes.goal#fri,1,0,0,0,0


'literature_nd'

...

,timeout,parse error,incorrect,crash,no output
no.goal#fri,0,2,0,0,0
no.goal#pit,0,2,0,0,0
yes.goal#fri,0,2,0,0,0
yes.goal#pit,0,2,0,0,0


'random_sd'

,timeout,parse error,incorrect,crash,no output
tool,,,,,


'random_nd'

,timeout,parse error,incorrect,crash,no output
no.buechenic,4,0,0,0,0
no.goal#fri,3,0,0,0,0
no.goal#pit,4,0,0,0,0
yes.autfilt,1,0,0,0,0
yes.buechenic,4,0,0,0,0
yes.goal#fri,4,0,0,0,0
yes.goal#pit,5,0,0,0,0
yes.ncsb#best,1,0,0,0,0
yes.ncsb#pldi,1,0,0,0,0
yes.ncsb#spot,1,0,0,0,0
